## Category_2よってamountの絶対値あたりの価値は違うという仮説で、特徴をリフトにする

In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features
import datetime
import time
from tqdm import tqdm
import sys
import glob

os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

### Data Load

In [5]:
# regist_termをもたせる
df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')
train_test = pd.concat([df_train, df_test], axis=0)
train_test.set_index(key, inplace=True)
base = utils.read_df_pkl('../input/base_term*0*').set_index(key)[['hist_regist_term', 'new_regist_term']]
train_test = train_test.join(base)

df_hist = utils.read_df_pkl('../input/hist_clean*')
df_new = utils.read_df_pkl('../input/new_clean*')

df_train = utils.reduce_mem_usage(df_train)
df_test  = utils.reduce_mem_usage(df_test )
df_hist  = utils.reduce_mem_usage(df_hist )
df_new   = utils.reduce_mem_usage(df_new  )

100%|██████████| 3/3 [00:01<00:00,  2.19it/s]


Mem. usage decreased to  5.58 Mb (48.2% reduction)
Mem. usage decreased to  3.18 Mb (43.8% reduction)
Mem. usage decreased to 2554.26 Mb (0.0% reduction)
Mem. usage decreased to 183.47 Mb (0.0% reduction)


In [4]:
#========================================================================
# 登録より前に購買が発生しているユーザーかどうかわかるdiff term 
prefix = '555_elo'
base = utils.read_df_pkl('../input/base_term*0*')
base[['hist_regist_term','hist_personal_term']].groupby(['hist_regist_term','hist_personal_term']).size()
base["diff_regist_term_hist_term"] = base['hist_regist_term'] - base['hist_personal_term']
utils.to_pkl_gzip(obj=base["diff_regist_term_hist_term"], path=f'../features/1_first_valid/{prefix}_diff_regist_term_hist_term')
#========================================================================

100%|██████████| 3/3 [00:00<00:00, 18.93it/s]


In [7]:
#========================================================================
# CLV
df_hist = df_hist[['card_id','purchase_date','purchase_amount']]

## Time
from datetime import datetime

z = df_hist.groupby('card_id')['purchase_date'].max().reset_index()
q = df_hist.groupby('card_id')['purchase_date'].min().reset_index()

z.columns = ['card_id', 'Max']
q.columns = ['card_id', 'Min']

## Extracting current timestamp
now = datetime.now()
curr_date = now.strftime("%m-%d-%Y, %H:%M:%S")
curr_date = pd.to_datetime(curr_date)

rec = pd.merge(z,q, how = 'left',on = 'card_id')
rec['Min'] = pd.to_datetime(rec['Min'])
rec['Max'] = pd.to_datetime(rec['Max'])

## Time value 
rec['Recency'] = (curr_date - rec['Max']).astype('timedelta64[D]') ## current date - most recent date

## Recency value
rec['Time'] = (rec['Max'] - rec['Min']).astype('timedelta64[D]') ## Age of customer, MAX - MIN

rec = rec[['card_id','Time','Recency']]

## Frequency
freq = df_hist.groupby('card_id').size().reset_index()
freq.columns = ['card_id', 'Frequency']
## Monitary
mon = df_hist.groupby('card_id')['purchase_amount'].sum().reset_index()
mon.columns = ['card_id', 'Monitary']

final = pd.merge(freq,mon,how = 'left', on = 'card_id')
final = pd.merge(final,rec,how = 'left', on = 'card_id')

final['historic_CLV'] = final['Frequency'] * final['Monitary'] 
final['AOV'] = final['Monitary']/final['Frequency'] ## AOV - Average order value (i.e) total_purchase_amt/total_trans
final['Predictive_CLV'] = final['Time']*final['AOV']*final['Monitary']*final['Recency'] 

for col in ['AOV', 'Predictive_CLV']:
    feature = final[col].fillna(-1).values.astype('float32')
    print(feature.shape)
    utils.to_pkl_gzip(obj=feature, path=f'../features/1_first_valid/{prefix}_{col}')
#========================================================================

,card_id,Time,Recency
0,C_ID_00007093c1,377.0,343.0
1,C_ID_0001238066,151.0,342.0
2,C_ID_0001506ef0,398.0,352.0
3,C_ID_0001793786,283.0,461.0
4,C_ID_000183fdda,202.0,344.0


In [7]:
#========================================================================
# Featureをnew_regist_term別にliftにする
# train_testの粒度でliftを作る
win_path = f'../features/4_winner/*.gz'
win_path_list = glob.glob(win_path)
base = utils.read_df_pkl('../input/base_term*0*')
feature_list = utils.parallel_load_data(path_list=win_path_list)

lift_level = 'hist_regist_term'

df_feat = pd.concat(feature_list, axis=1)
df_feat = pd.concat([base, df_feat], axis=1)

100%|██████████| 3/3 [00:00<00:00,  7.52it/s]


In [ ]:
feat_cols = [col for col in df_feat.columns if col.count('@') and not(col.count('lift')) and not(col.count('114_'))]
level_list = np.unique(df_feat[lift_level].values)
tmp_list = []
df_list = []

for col in tqdm(feat_cols):
    for level in level_list:
        tmp = df_feat[df_feat[lift_level]==level][[key, col]]
        base_avg = tmp[col].mean()
        tmp[col] = tmp[col] / base_avg
        tmp.rename(columns={col:f'lift_{col}'}, inplace=True)
        tmp_list.append(tmp)
        
    df_level = pd.concat(tmp_list, axis=0)
#     df_level[lift_level] = level
    df_level.set_index(key, inplace=True)
    df_list.append(df_level)
        
df_lift = pd.concat(df_list, axis=1)
df_lift = df_lift.reset_index().set_index(key)
print(df_lift.shape)

df_lift.head()
#========================================================================

 18%|█▊        | 27/148 [01:09<12:14,  6.07s/it]